In [1]:
import pandas as pd
import numpy as np
import re
import os
import shutil
from ptm_dict import modres

In [9]:
from datetime import datetime
time_stamp = datetime.today().strftime('%Y-%m-%d')

In [2]:
# Downstream code heavily depends on which input dataframe you will use, since it uses the PDB ID and mutation data to align the predictions with the SKEMPI 2.0 entries

#skempi2 = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/skempi2_final_final.csv')
# skempi2.reset_index(drop=True, inplace=True)
# skempi2.drop(columns=['index', 'case_ID'])
skempiMAX = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/skempiMAX.csv')
# #skempi2_avg_nb = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/skempi2_nb.csv')
# skempi2_raw = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/skempi_v2.csv', sep=';')
# skempi2_avg_nb = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/skempi2_nb.csv')

In [ ]:
# exp_cart_ddG_all = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/cart_pipeline_full_struct.csv')
# flex_ddg_gam = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/flex_ddg_gam_skempi2.csv')
# flex_ddg_reg = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/flex_ddg_reg_skempi2.csv')
# exp_cart_ddG_all_flex = pd.merge(exp_cart_ddG_all, flex_ddg_reg[['#Pdb', 'Mutation(s)_PDB', 'total_score']], on=['#Pdb', 'Mutation(s)_PDB'], how='left')

In [3]:
output_path = '/lustre/hpc/sbinlab/panf/SKEMPIv2_benchmark/cartesian_runs_folder/'
input_path = '/lustre/hpc/sbinlab/panf/SKEMPIv2_benchmark/benchmark_inputs/raw_PDBs'

In [4]:
skempi2 = skempiMAX.copy()

In [5]:
## Loading the data

# Creating lists to store cart_ddG output stored as dfs
ddG_df_list_complex = list()
# Creating lists to store names of PDBs with empty output folders
no_output = list()
is_output = list()
# Going through contents of run dir
for folder in os.listdir(output_path):
    # If item is not a slurm file or a Jupyter checkpoint file, it's a folder with cart_ddG run files
    # Save path to 'output' folder in cart_ddG PDB run folder. This is the path to ddG_complex calculations.
    if not folder.startswith(('.ipynb_', 'slurm')):
        path_to_output = os.path.join(output_path, folder, "output")
        path_to_pdb = os.path.join(output_path, folder)
        # If 'output' folder is not empty, read prism file, add PDBFileID column (by folder name), append list of dfs
        if len(os.listdir(path_to_output)) > 0:
            for file in os.listdir(path_to_output):
                # if file.startswith('prism_rosetta_'):
                #     if not file.endswith('-gap.txt'):
                #         path_to_prism = os.path.join(path_to_output, file)
                #         x = pd.read_csv(path_to_prism, sep=" ", comment='#')
                #         x['PDB_ID'] = folder
                #         ddG_df_list_complex.append(x)
                if file.endswith('_full_interface_IAM_single_load.csv'):
                # if 'IAM' in file:
                    path_to_prism = os.path.join(path_to_output, file)
                    x = pd.read_csv(path_to_prism)
                    x['PDB_ID'] = folder
                    ddG_df_list_complex.append(x)
                    is_output.append(folder)
                        
        # If 'output' folder is empty, add PDB name to no_output list
        else:
            no_output.append(folder)

# Concatenate list of dfs into one for ddG_complex
all_cases_muts = pd.concat(ddG_df_list_complex)
variants = all_cases_muts[~all_cases_muts.variant.str.contains(r'=')].copy()
variants.reset_index(drop=True, inplace=True)
no_output.sort()

print('No output for the following PDB IDs: ', no_output)
print('The following PDB IDs from SKEMPI v2 are not in the output: ')
for pdb_id in skempi2.PDB_ID.unique():
# for pdb_id in skempi2_raw['#Pdb'].str[:4].unique():
    if not pdb_id in os.listdir(output_path):
        print(pdb_id)
    if not pdb_id in is_output:
        print('IA problem: ', pdb_id)

No output for the following PDB IDs:  []
The following PDB IDs from SKEMPI v2 are not in the output: 
3SE3
IA problem:  3SE3
3SE4
IA problem:  3SE4
2C5D
IA problem:  2C5D


In [6]:
# Renumbering the residues from Rosetta numeration to chain + num for the whole PDB

dict_of_dicts = {}
for folder_pdb in os.listdir(output_path):
    if not (folder_pdb.endswith('.ipynb_checkpoints')) and not (folder_pdb.startswith('slurm-')):
        for item in os.listdir(os.path.join(output_path, folder_pdb)):
            if item.endswith('.sbatch'):
                if not item.endswith(('IA_calc.sbatch', 'IA_calc_wt.sbatch', 'IA_calc_satmut.sbatch')):
                    chains = item.split('.')[0].split('_')[2]
        
        pdb_path = os.path.join(output_path, folder_pdb, 'prepare', 'input', 'input.pdb')
        
        pdb_file = open(pdb_path, 'r')
        lines = pdb_file.readlines()
        raa_prev = ''
        rn_pres = ''
        pos_chain = {}
        pos_add = 1
        for line in lines:
            if folder_pdb == '1DAN':
                if line.startswith('ATOM'):
                    if line[21] in chains:
                        if (line[17:20] != raa_prev)|(line[22:27] != rn_pres):
                            chain_num = line[21] + ' ' + line[22:27].strip(' ').lower()
                            pos_chain[chain_num] = pos_add
                            raa_prev = line[17:20]
                            rn_pres = line[22:27]
                            pos_add += 1
                            
            else:
                if line.startswith('ATOM') | line.startswith('HETATM'):
                    if line[17:20] in modres.keys():
                        if line[21] in chains:
                            if (line[17:20] != raa_prev)|(line[22:27] != rn_pres):
                                chain_num = line[21] + ' ' + line[22:27].strip(' ').lower()
                                pos_chain[chain_num] = pos_add
                                raa_prev = line[17:20]
                                rn_pres = line[22:27]
                                pos_add += 1
        pdb_file.close()
        dict_of_dicts[folder_pdb] = pos_chain

In [7]:
# Renumbering resfile notation to prism output notation

dict_mult_interface = {'2C5D_AB_CD':'2C5D_ABCD_ABCD', '2C5D_A_C':'2C5D_C_AC',
                       '3SE3_B_C':'3SE3_B_BC', '3SE3_B_A':'3SE3_BA_BA',
                       '3SE4_B_A':'3SE4_AB_BA', '3SE4_B_C':'3SE4_CB_BC'}

dict_mult_interface_rev = {'2C5D_ABCD_ABCD':'2C5D_AB_CD', '2C5D_C_AC':'2C5D_A_C',
                       '3SE3_B_BC':'3SE3_B_C', '3SE3_BA_BA':'3SE3_B_A',
                       '3SE4_AB_BA':'3SE4_B_A', '3SE4_CB_BC':'3SE4_B_C'}

# Go through the dataframe row by row. Get PDB id for every case and mutations as a list. Define list to append reformated mutations to.
reformat_muts = list()
for index, row in skempi2.iterrows():
# for index, row in skempi2_raw.iterrows():
    # pdb_list = list()
    # pdb = row['PDB_ID']
    pdb = row['#Pdb'][0:4]
    pdb_c = row['#Pdb']
    
    if pdb.startswith(('2C5D', '3SE3', '3SE4')):
        pdb = dict_mult_interface[pdb_c]
    #     for file in os.listdir(os.path.join(input_path, pdb)):
    #         if file.endswith('.txt'):
    #             pdb_list.append(file.split('.')[0])
    # else:
    #     pdb_list.append(row['PDB_ID'])

    # for pdb in pdb_list:
                
    list_mut = row['Mutation(s)_PDB'].split(',')
    # string_full_mut = ''
    list_full_mut = list()
    count = 0
# Go through mutations and append reformated mutations to the list
    for mut in list_mut:
        count += 1
        pos = mut[2:-1]
        chain = mut[1]
        sub = mut[-1]     
        wt = mut[0]
        word = chain + ' ' + pos
        # For every mutation, look in the dictionary for chain plus position and translate to Rosetta numeration
        try:
            translation = str(dict_of_dicts[pdb][word])
        except:
            print(pdb, word, '-- zero occupancy issue')
        prism_format = wt + translation + sub
        if prism_format not in list_full_mut:
            list_full_mut.append(prism_format)
    list_full_mut.sort(key = lambda x: int(x[1:-1]))
    delim = ":"
    # using map to convert each element to string
    temp = list(map(str, list_full_mut))
    # join() used to join with delimiter
    string_full_mut = delim.join(temp)
    reformat_muts.append(string_full_mut)
# skempi2_raw['variant'] = reformat_muts
skempi2['variant'] = reformat_muts

1JCK B 103 -- zero occupancy issue
1JCK B 102 -- zero occupancy issue
1AK4 D 496 -- zero occupancy issue


In [11]:
# skempi2_raw['PDB_ID'] = skempi2_raw['#Pdb'].str[:4]
# exp_cart_ddG_all = pd.merge(variants, skempi2_raw, on=['PDB_ID', 'variant'], how='right')
exp_cart_ddG_all = pd.merge(variants, skempi2, on=['PDB_ID', 'variant'], how='right')

for index, row in exp_cart_ddG_all.iterrows():
    if row['#Pdb'] in dict_mult_interface.keys():
        
        ddG_value = variants.loc[(variants['PDB_ID'] == dict_mult_interface[row['#Pdb']]
                     )&(variants['variant'] == row['variant'])]['mean_ddG'].values[0]
        std_value = variants.loc[(variants['PDB_ID'] == dict_mult_interface[row['#Pdb']]
                     )&(variants['variant'] == row['variant'])]['std_ddG'].values[0]

        # print('mean ddG', exp_cart_ddG_all.iloc[index, 1], ddG_value)
        # print('std', exp_cart_ddG_all.iloc[index, 2], std_value)
        # count += 1
        exp_cart_ddG_all.iloc[index, 1] = ddG_value
        exp_cart_ddG_all.iloc[index, 2] = std_value
        
#exp_cart_ddG_all.loc[exp_cart_ddG_all['mean_ddG']>15, 'mean_ddG'] = 15
exp_cart_ddG_all['N_mut'] = exp_cart_ddG_all['variant'].str.split(':').str.len()
exp_cart_ddG_all['N_mut'] = exp_cart_ddG_all['N_mut'].apply(lambda x: '3+' if x >= 3 else x)
exp_cart_ddG_all['N_mut'] = exp_cart_ddG_all['N_mut'].apply(str)
#exp_cart_ddG_all.to_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/cart_ddg_skempiMAX.csv', index=False)
exp_cart_ddG_all.to_csv(f'/groups/sbinlab/panf/SKEMPIv2_benchmark/skempiMAX_cart_ddg_IA_{time_stamp}.csv', index=False)

In [16]:
exp_cart_ddG_all.loc[exp_cart_ddG_all['mean_ddG']>15]

,variant,mean_ddG,std_ddG,mean_ddG_infc,std_ddG_infc,PDB_ID,Entry,#Pdb,Mutation(s)_PDB,Mutation(s)_cleaned,...,dG_bind_MUT,ddG_bind,LChains,RChains,Chains,small-to-large,type_of_change_infc,type_of_change_aa,interaction_strength,N_mut
921,I12W,15.660345,0.000000,0.158247,0.000000,1FC2,1073,1FC2_C_D,IC135W,IC12W,...,-7.530779,3.137594,C,D,CD,True,infc to infc,same type,moderate,1
2887,A233V:T235S:L236P:R239M:N246S:G250N:N254D:N257...,16.742529,0.195345,9.199784,0.011174,1PPF,3437,1PPF_E_I,"AI15V,GI32N,LI18P,NI28S,NI36D,NI39S,RI21M,SI51...","AI15V,TI17S,LI18P,RI21M,NI28S,GI32N,NI36D,NI39...",...,-5.892045,7.271511,E,I,EI,False,mixed,mixed,strong,3+
3304,A478S:L479G:P480T:G482K:I634V:H635G:P636A:E637...,19.869655,0.175875,6.189682,0.399818,4MNQ,4047,4MNQ_ABC_DE,"AD94S,EE53G,GD98K,HE51G,IE50V,LD95G,PD96T,PE52...","IE49V,HE50G,PE51A,EE52G,YE53I,AD93S,LD94G,PD95...",...,-6.043025,5.818547,ABC,DE,ABCDE,False,mixed,mixed,moderate,3+
3305,A478S:L479G:P480T:G482K,15.883678,0.118947,1.731139,0.250695,4MNQ,4048,4MNQ_ABC_DE,"AD94S,GD98K,LD95G,PD96T","AD93S,LD94G,PD95T,GD97K",...,-10.938859,0.922713,ABC,DE,ABCDE,False,mixed,mixed,moderate,3+
3306,A478S:L479G:P480T:G482K:H635W:P636E:E637F:Y638E,19.922069,0.281738,4.072345,0.434511,4MNQ,4049,4MNQ_ABC_DE,"AD94S,EE53F,GD98K,HE51W,LD95G,PD96T,PE52E,YE54E","HE50W,PE51E,EE52F,YE53E,AD93S,LD94G,PD95T,GD97K",...,-10.964333,0.897238,ABC,DE,ABCDE,False,mixed,mixed,moderate,3+
3313,L382A:A478S:L479G:P480T:G482K,16.138276,0.027635,3.234741,0.041150,4MNQ,4059,4MNQ_ABC_DE,"AD94S,GD98K,LC6A,LD95G,PD96T","LC6A,AD93S,LD94G,PD95T,GD97K",...,-6.761361,5.100211,ABC,DE,ABCDE,False,mixed,mixed,moderate,3+
3314,L382A:A478S:L479G:P480T:G482K:H635W:P636E:E637...,17.522644,1.401578,2.010380,0.544603,4MNQ,4060,4MNQ_ABC_DE,"AD94S,EE53F,GD98K,HE51W,LC6A,LD95G,PD96T,PE52E...","LC6A,HE50W,PE51E,EE52F,YE53E,AD93S,LD94G,PD95T...",...,-6.437705,5.423867,ABC,DE,ABCDE,False,mixed,mixed,moderate,3+
3390,L55P:L170P:L288P:L402P,41.649885,0.047531,-0.001544,0.000037,1QAB,4152,1QAB_ABCD_E,"LA55P,LB55P,LC55P,LD55P","LA55P,LB46P,LC46P,LD46P",...,-8.286026,0.369326,ABCD,E,ABCDE,False,infc to non-infc,hydrophobic to special,moderate,3+
4214,K613S:D614G:T637S:N638E:G683V:D685V:D824N:P826...,15.687816,0.022489,1.189204,0.019433,3BE1,5195,3BE1_HL_A,"DH31G,DH98V,DL27aN,GH96V,IL29A,KH30S,NH54E,PL2...","DL28N,PL30A,RL31K,SL32T,TL97S,TL98S,KH30S,DH31...",...,-13.447030,-2.772878,HL,A,HLA,False,mixed,mixed,moderate,3+
4641,L477A:V478A:L1069A:V1070A:L1651A:V1652A:L2147A...,22.172414,0.338773,9.352973,0.056545,3VR6,5753,3VR6_ABCDEF_GH,"LA476A,LB476A,LC476A,LD389A,LE389A,LF389A,VA47...","LA477A,LB482A,LC472A,VA478A,VB483A,VC473A,LD38...",...,-12.164076,-0.170362,ABCDEF,GH,ABCDEFGH,False,infc to non-infc,same type,moderate,3+


In [ ]:
exp_cart_ddG_all_nb = exp_cart_ddG_all.copy()
exp_cart_ddG_all_nb.loc[exp_cart_ddG_all_nb['no_proper_value']=='no binding MUT', 'dG_bind_MUT'] = -1.5
exp_cart_ddG_all_nb['ddG_bind'] = exp_cart_ddG_all_nb['dG_bind_MUT'].values - exp_cart_ddG_all_nb['dG_bind_WT'].values

exp_cart_ddG_all_no_nan = exp_cart_ddG_all.copy().dropna(subset=['ddG_bind', 'mean_ddG', 'mean_ddG_infc'])
#exp_cart_ddG_all_no_nan = exp_cart_ddG_all_no_nan.loc[exp_cart_ddG_all_no_nan['no_proper_value']=='Regular']

In [ ]:
exp_cart_ddG_all_no_nan.loc[exp_cart_ddG_all_no_nan['Hold_out_type'].isna(), 'Hold_out_type'] = 'None'

In [ ]:
exp_cart_ddG_all_no_nan.Hold_out_type.unique()

In [ ]:
exp_flex_ddG = pd.read_csv('flex_ddg_both_skempi2_std.csv')
exp_cart_ddg_ia = pd.read_csv('skempiMAX_cart_ddg_IA.csv')
skempiMAX = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/skempiMAX.csv')
# SkempiMAX_cart_flex = pd.read_csv('/groups/sbinlab/panf/SKEMPIv2_benchmark/skempiMAX_cart_ddg_IA_Flex_ddG_2025-10-25.csv')
SkempiMAX_cart_flex = pd.merge(exp_cart_ddg_ia, exp_flex_ddG[['#Pdb', 'case_ID',
                                                   'Mutation(s)_PDB', 'flex_ddg',
                                                   'flex_ddg_gam', 'std']],
                     on =['#Pdb','Mutation(s)_PDB'], how='left')
SkempiMAX_cart_flex.to_csv(f'skempiMAX_cart_ddg_IA_Flex_ddG_{time_stamp}.csv', index=False)